## Build Silver Layer (SQL)

## What is the Silver layer?
**Silver** turns raw Bronze into **clean, conformed, join‑ready** tables suitable for analytics.


## Create `silver` database

In [ ]:
DROP DATABASE IF EXISTS silver;
CREATE DATABASE silver
  DEFAULT CHARACTER SET utf8mb4
  COLLATE utf8mb4_0900_ai_ci;
USE silver;

### Table: `silver.crm_cust_info`

**Changes & Why**
- **Trim** names; standardize **marital_status** and **gender**.
- **Deduplicate** on `cst_id` by latest `cst_create_date` (SCD‑1).
- Add **audit** column and **indexes**.

**Columns (post‑transform):**

| Column | Type | Nullable | Description |
|---|---|---|---|
| `cst_id` | `INT` | Yes | Natural key; dedup by latest create_date |
| `cst_key` | `VARCHAR(50)` | Yes | Business key to map ERP |
| `cst_firstname` | `VARCHAR(50)` | Yes | Trimmed |
| `cst_lastname` | `VARCHAR(50)` | Yes | Trimmed |
| `cst_marital_status` | `VARCHAR(50)` | Yes | Standardized: Single/Married/n/a |
| `cst_gndr` | `VARCHAR(50)` | Yes | Standardized: Male/Female/n/a |
| `cst_create_date` | `DATE` | Yes | Original create date |
| `dwh_create_date` | `DATETIME` | Yes | Audit timestamp |

#### Create table `silver.crm_cust_info` (DDL)

In [ ]:
CREATE TABLE IF NOT EXISTS crm_cust_info (
  cst_id INT,
  cst_key VARCHAR(50),
  cst_firstname VARCHAR(50),
  cst_lastname VARCHAR(50),
  cst_marital_status VARCHAR(50),
  cst_gndr VARCHAR(50),
  cst_create_date DATE,
  dwh_create_date DATETIME DEFAULT CURRENT_TIMESTAMP,
  INDEX ix_crm_cust_info_key (cst_key),
  INDEX ix_crm_cust_info_id (cst_id)
) ENGINE=InnoDB;

#### Transform Bronze → Silver for `crm_cust_info` (and preview)

In [ ]:
TRUNCATE TABLE silver.crm_cust_info;
INSERT INTO silver.crm_cust_info (
  cst_id, cst_key, cst_firstname, cst_lastname,
  cst_marital_status, cst_gndr, cst_create_date
)
SELECT cst_id,
       cst_key,
       TRIM(cst_firstname),
       TRIM(cst_lastname),
       CASE UPPER(TRIM(cst_marital_status))
         WHEN 'S' THEN 'Single'
         WHEN 'M' THEN 'Married'
         ELSE 'n/a'
       END,
       CASE UPPER(TRIM(cst_gndr))
         WHEN 'F' THEN 'Female'
         WHEN 'M' THEN 'Male'
         ELSE 'n/a'
       END,
       cst_create_date
FROM (
   SELECT b.*,
          ROW_NUMBER() OVER (PARTITION BY cst_id ORDER BY cst_create_date DESC) AS rn
   FROM bronze.crm_cust_info b
   WHERE cst_id IS NOT NULL
) t
WHERE rn = 1;
SELECT COUNT(*) AS rows_loaded FROM silver.crm_cust_info;
SELECT * FROM silver.crm_cust_info LIMIT 10;

### Table: `silver.crm_prd_info`

**Changes & Why**
- **Derive** `cat_id`; **decode** `prd_line` labels.
- **Effective dating** via `LEAD(prd_start_dt) - 1 day`.
- Add **audit** column and **indexes**.

**Columns (post‑transform):**

| Column | Type | Nullable | Description |
|---|---|---|---|
| `prd_id` | `INT` | Yes | Natural product id |
| `cat_id` | `VARCHAR(50)` | Yes | Derived from prd_key |
| `prd_key` | `VARCHAR(50)` | Yes | Clean business key |
| `prd_nm` | `VARCHAR(50)` | Yes | Product name |
| `prd_cost` | `INT` | Yes | Unit cost |
| `prd_line` | `VARCHAR(50)` | Yes | Decoded label |
| `prd_start_dt` | `DATE` | Yes | Start date |
| `prd_end_dt` | `DATE` | Yes | Effective end date via LEAD-1 |
| `dwh_create_date` | `DATETIME` | Yes | Audit timestamp |

#### Create table `silver.crm_prd_info` (DDL)

In [ ]:
CREATE TABLE IF NOT EXISTS crm_prd_info (
  prd_id INT,
  cat_id VARCHAR(50),
  prd_key VARCHAR(50),
  prd_nm VARCHAR(50),
  prd_cost INT,
  prd_line VARCHAR(50),
  prd_start_dt DATE,
  prd_end_dt DATE,
  dwh_create_date DATETIME DEFAULT CURRENT_TIMESTAMP,
  INDEX ix_crm_prd_key (prd_key),
  INDEX ix_crm_cat_id (cat_id)
) ENGINE=InnoDB;

#### Transform Bronze → Silver for `crm_prd_info` (and preview)

In [ ]:
TRUNCATE TABLE silver.crm_prd_info;
INSERT INTO silver.crm_prd_info (
  prd_id, cat_id, prd_key, prd_nm, prd_cost, prd_line, prd_start_dt, prd_end_dt
)
SELECT
  prd_id,
  REPLACE(SUBSTRING(prd_key, 1, 5), '-', '_') AS cat_id,
  SUBSTRING(prd_key, 7) AS prd_key,
  prd_nm,
  COALESCE(prd_cost, 0),
  CASE UPPER(TRIM(prd_line))
    WHEN 'M' THEN 'Mountain'
    WHEN 'R' THEN 'Road'
    WHEN 'S' THEN 'Other Sales'
    WHEN 'T' THEN 'Touring'
    ELSE 'n/a'
  END,
  DATE(prd_start_dt),
  DATE( (LEAD(prd_start_dt) OVER (PARTITION BY SUBSTRING(prd_key, 7) ORDER BY prd_start_dt)) - INTERVAL 1 DAY )
FROM bronze.crm_prd_info;
SELECT COUNT(*) AS rows_loaded FROM silver.crm_prd_info;
SELECT * FROM silver.crm_prd_info LIMIT 10;

### Table: `silver.crm_sales_details`

**Changes & Why**
- **Parse** dates from `YYYYMMDD`; **recompute** sales when inconsistent; **backfill** price.
- Add **audit** column and **indexes**.

**Columns (post‑transform):**

| Column | Type | Nullable | Description |
|---|---|---|---|
| `sls_ord_num` | `VARCHAR(50)` | Yes | Order number |
| `sls_prd_key` | `VARCHAR(50)` | Yes | Product key |
| `sls_cust_id` | `INT` | Yes | Customer id |
| `sls_order_dt` | `DATE` | Yes | Parsed from YYYYMMDD |
| `sls_ship_dt` | `DATE` | Yes | Parsed from YYYYMMDD |
| `sls_due_dt` | `DATE` | Yes | Parsed from YYYYMMDD |
| `sls_sales` | `INT` | Yes | Recomputed if inconsistent |
| `sls_quantity` | `INT` | Yes | Units |
| `sls_price` | `INT` | Yes | Backfilled if missing |
| `dwh_create_date` | `DATETIME` | Yes | Audit timestamp |

#### Create table `silver.crm_sales_details` (DDL)

In [ ]:
CREATE TABLE IF NOT EXISTS crm_sales_details (
  sls_ord_num VARCHAR(50),
  sls_prd_key VARCHAR(50),
  sls_cust_id INT,
  sls_order_dt DATE,
  sls_ship_dt DATE,
  sls_due_dt DATE,
  sls_sales INT,
  sls_quantity INT,
  sls_price INT,
  dwh_create_date DATETIME DEFAULT CURRENT_TIMESTAMP,
  INDEX ix_sales_ord (sls_ord_num),
  INDEX ix_sales_prd (sls_prd_key),
  INDEX ix_sales_cust (sls_cust_id)
) ENGINE=InnoDB;

#### Transform Bronze → Silver for `crm_sales_details` (and preview)

In [ ]:
TRUNCATE TABLE silver.crm_sales_details;
INSERT INTO silver.crm_sales_details (
  sls_ord_num, sls_prd_key, sls_cust_id, sls_order_dt, sls_ship_dt, sls_due_dt,
  sls_sales, sls_quantity, sls_price
)
SELECT 
  sls_ord_num,
  sls_prd_key,
  sls_cust_id,
  CASE WHEN sls_order_dt = 0 OR LENGTH(sls_order_dt) != 8 THEN NULL
       ELSE STR_TO_DATE(CAST(sls_order_dt AS CHAR), '%Y%m%d') END,
  CASE WHEN sls_ship_dt = 0 OR LENGTH(sls_ship_dt) != 8 THEN NULL
       ELSE STR_TO_DATE(CAST(sls_ship_dt AS CHAR), '%Y%m%d') END,
  CASE WHEN sls_due_dt = 0 OR LENGTH(sls_due_dt) != 8 THEN NULL
       ELSE STR_TO_DATE(CAST(sls_due_dt AS CHAR), '%Y%m%d') END,
  CASE 
    WHEN sls_sales IS NULL OR sls_sales <= 0 OR sls_sales <> sls_quantity * ABS(sls_price)
      THEN sls_quantity * ABS(sls_price)
    ELSE sls_sales
  END,
  sls_quantity,
  CASE 
    WHEN sls_price IS NULL OR sls_price <= 0 THEN NULLIF(sls_sales,0) / NULLIF(sls_quantity,0)
    ELSE sls_price
  END
FROM bronze.crm_sales_details;
SELECT COUNT(*) AS rows_loaded FROM silver.crm_sales_details;
SELECT * FROM silver.crm_sales_details LIMIT 10;

### Table: `silver.erp_cust_az12`

**Changes & Why**
- **Strip** `NAS` prefix; **validate** `bdate`; **normalize** gender.

**Columns (post‑transform):**

| Column | Type | Nullable | Description |
|---|---|---|---|
| `cid` | `VARCHAR(50)` | Yes | NAS prefix stripped where present |
| `bdate` | `DATE` | Yes | Future dates set to NULL |
| `gen` | `VARCHAR(50)` | Yes | Normalized gender |
| `dwh_create_date` | `DATETIME` | Yes | Audit timestamp |

#### Create table `silver.erp_cust_az12` (DDL)

In [ ]:
CREATE TABLE IF NOT EXISTS erp_cust_az12 (
  cid VARCHAR(50),
  bdate DATE,
  gen VARCHAR(50),
  dwh_create_date DATETIME DEFAULT CURRENT_TIMESTAMP,
  INDEX ix_custaz_cid (cid)
) ENGINE=InnoDB;

#### Transform Bronze → Silver for `erp_cust_az12` (and preview)

In [ ]:
TRUNCATE TABLE silver.erp_cust_az12;
INSERT INTO silver.erp_cust_az12 (cid, bdate, gen)
SELECT
  CASE WHEN cid LIKE 'NAS%%' THEN SUBSTRING(cid, 4) ELSE cid END,
  CASE WHEN bdate > CURRENT_DATE THEN NULL ELSE bdate END,
  CASE
     WHEN UPPER(TRIM(gen)) IN ('F','FEMALE') THEN 'Female'
     WHEN UPPER(TRIM(gen)) IN ('M','MALE')   THEN 'Male'
     ELSE 'n/a'
  END
FROM bronze.erp_cust_az12;
SELECT COUNT(*) AS rows_loaded FROM silver.erp_cust_az12;
SELECT * FROM silver.erp_cust_az12 LIMIT 10;

### Table: `silver.erp_loc_a101`

**Changes & Why**
- **Normalize** `cid` and **map** country labels.

**Columns (post‑transform):**

| Column | Type | Nullable | Description |
|---|---|---|---|
| `cid` | `VARCHAR(50)` | Yes | Hyphens removed |
| `cntry` | `VARCHAR(50)` | Yes | Mapped labels (DE/US) or n/a |
| `dwh_create_date` | `DATETIME` | Yes | Audit timestamp |

#### Create table `silver.erp_loc_a101` (DDL)

In [ ]:
CREATE TABLE IF NOT EXISTS erp_loc_a101 (
  cid VARCHAR(50),
  cntry VARCHAR(50),
  dwh_create_date DATETIME DEFAULT CURRENT_TIMESTAMP,
  INDEX ix_loc_cid (cid)
) ENGINE=InnoDB;

#### Transform Bronze → Silver for `erp_loc_a101` (and preview)

In [ ]:
TRUNCATE TABLE silver.erp_loc_a101;
INSERT INTO silver.erp_loc_a101 (cid, cntry)
SELECT
  REPLACE(cid, '-', ''),
  CASE
     WHEN TRIM(cntry) = 'DE' THEN 'Germany'
     WHEN TRIM(cntry) IN ('US','USA') THEN 'United States'
     WHEN TRIM(cntry) = '' OR cntry IS NULL THEN 'n/a'
     ELSE TRIM(cntry)
  END
FROM bronze.erp_loc_a101;
SELECT COUNT(*) AS rows_loaded FROM silver.erp_loc_a101;
SELECT * FROM silver.erp_loc_a101 LIMIT 10;

### Table: `silver.erp_px_cat_g1v2`

**Changes & Why**
- **Carry over** mapping with audit + index.

**Columns (post‑transform):**

| Column | Type | Nullable | Description |
|---|---|---|---|
| `id` | `VARCHAR(50)` | Yes | Category id |
| `cat` | `VARCHAR(50)` | Yes | Category |
| `subcat` | `VARCHAR(50)` | Yes | Subcategory |
| `maintenance` | `VARCHAR(50)` | Yes | Maintenance attribute |
| `dwh_create_date` | `DATETIME` | Yes | Audit timestamp |

#### Create table `silver.erp_px_cat_g1v2` (DDL)

In [ ]:
CREATE TABLE IF NOT EXISTS erp_px_cat_g1v2 (
  id VARCHAR(50),
  cat VARCHAR(50),
  subcat VARCHAR(50),
  maintenance VARCHAR(50),
  dwh_create_date DATETIME DEFAULT CURRENT_TIMESTAMP,
  INDEX ix_cat_id (id)
) ENGINE=InnoDB;

#### Transform Bronze → Silver for `erp_px_cat_g1v2`

In [ ]:
TRUNCATE TABLE silver.erp_px_cat_g1v2;
INSERT INTO silver.erp_px_cat_g1v2 (id, cat, subcat, maintenance)
SELECT id, cat, subcat, maintenance FROM bronze.erp_px_cat_g1v2;
SELECT COUNT(*) AS rows_loaded FROM silver.erp_px_cat_g1v2;
SELECT * FROM silver.erp_px_cat_g1v2 LIMIT 10;